# Parkplatzdaten Reinigung
- automatisierte Reinigung von Parkplatzdaten
- inklusive Spezialfall bei 2018

In [ ]:
# Man muss für jedes Jahr den entsrechenden Index setzen!
# 0 bis 3
_file_index = 0

In [3]:
import_dir = "PPD_ungereinigt/"
export_dir = "PPD_gereinigt/"
files_list = [  'Einlaeufe2017.csv',
                'Einlaeufe2018.csv', 
                'Einlaeufe2019.csv', 
                'Einlaeufe2020.csv'
                ]

In [2]:
import pandas as pd
import numpy as np
from ipywidgets import interactive

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [4]:
import_dirs_list = [import_dir+f for f in files_list]

In [5]:
#'Daten/Einlaeufe2020.csv'
def Vorbereitung_Phase_I(filepath, key, encoding):
    # -> normalisierte Datentypen, Schlüsselsetzung
    def csv_to_dataframe(filepath):
        return pd.read_csv(filepath_or_buffer=filepath, sep =';', encoding=encoding)
    
    def transform(df, key):
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], dayfirst=True)
        df = df.dropna(how='all', axis=1)
        df = df.set_index(key)
        t_df = df.sort_index()
        return t_df
    
    df_AK = transform(csv_to_dataframe(filepath), key).filter(like = 'AK') 
    df_AK[~df_AK.apply(pd.to_numeric, errors='coerce').notnull().all(1)]

    return df_AK.apply(pd.to_numeric, errors='coerce')

In [6]:
def Vorbereitung_Phase_II(df: pd.DataFrame):
    # -> Prozentanzahl
    def max_limit(plaetze):
        def foo(belegt):
            return belegt/plaetze
        return foo
    
    max_p = {
        'P1AK': max_limit(640),
        'P2AK': max_limit(540),
        'P3AK': max_limit(540),
        'P41AK': max_limit(670),
        'P42AK': max_limit(670),
        'P5AK': max_limit(500),
        'P61AK': max_limit(290),
        'P62AK': max_limit(290),
        'P63AK': max_limit(290),
        'P7AK': max_limit(290),
        'P8AK': max_limit(160)
    }
    
    for c in df.columns:
        df[c].apply(func=max_p[c])

    return df

In [7]:
def Vorbereitung(filepath, key="TimeStamp", encoding="utf-8"):
    a = Vorbereitung_Phase_I(filepath, key, encoding)
    return Vorbereitung_Phase_II(a)

In [8]:
def visualize(df, **kwargs):
    df.iplot(**kwargs)

In [14]:
def Reinigung(df):
    def analyze(column):
        pd.options.display.max_rows = 100
        print(df[column].sort_values(ascending=False))
        
    for c in df.columns:
        def a():
            analyze(c)
            visualize(df[c], title=c)
            s = input("Threashold: ")
            return s
        
        def setting_threshold(th):
            condition = df[c] <= th
            df[c].where(cond=condition, other=np.nan, inplace=True)
            ag = input("Adjust?: ")
            if "-" in ag:
                return False
            else:
                return True
            
        while(True):
            th = a()
            if th is not "-":
                while(True):
                    ag = setting_threshold(int(th))
                    if ag is False:
                        break
            else:
                break
    return df

In [15]:
def Speicherung(df):
    s_n = input("Select Name for Your Table: ")
    df.to_csv(export_dir + s_n + '.csv')

In [ ]:
df = Vorbereitung(import_dirs_list[_file_index], encoding="latin")

df = Reinigung(df)
Speicherung(df)

## Spezialfall
Bei 2018 sind die Datensätze verschoben

In [ ]:
#EIN SPEZIALFALL BEI 2018 TABELLE
df = pd.read_csv(filepath_or_buffer="new2020.csv", sep=",", encoding="utf-8")

#ÖFFNEN + INDEX SETZEN
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], dayfirst=True)
df = df.set_index("TimeStamp")

# CONDITION SETZEN
dd = df["TimeStamp"].apply(lambda x: x.replace(year=2018))
# BEARBEITEN
df["TimeStamp"] = np.where(df["TimeStamp"].dt.year == 2015, dd, df["TimeStamp"])